In [1]:
from newspaper import Article
from collectors.wikipedia import *
from collectors.google_news import *
from collectors.trends import *

import pandas as pd


/opt/anaconda3/envs/rag_llamaindex/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0
Device set to us

### Get Trending celebrities

In [ ]:
trendCollector = TrendCollector()
trending_celebs = await trendCollector.get_hottest_celebs(limit=100)
trending_celebs

07/19/2025 03:10:51 AM - Error converting news item: 'decoded_url'
07/19/2025 03:10:52 AM - Error converting news item: 'decoded_url'
07/19/2025 03:10:53 AM - Error converting news item: 'decoded_url'
07/19/2025 03:10:55 AM - Error converting news item: 'decoded_url'
07/19/2025 03:10:56 AM - Error converting news item: 'decoded_url'
07/19/2025 03:10:57 AM - Error converting news item: 'decoded_url'
07/19/2025 03:10:58 AM - Error converting news item: 'decoded_url'
07/19/2025 03:10:59 AM - Error converting news item: 'decoded_url'
07/19/2025 03:10:59 AM - Error converting news item: 'decoded_url'
07/19/2025 03:11:01 AM - Error converting news item: 'decoded_url'
07/19/2025 03:11:02 AM - Error converting news item: 'decoded_url'
07/19/2025 03:11:03 AM - Error converting news item: 'decoded_url'
07/19/2025 03:11:04 AM - Error converting news item: 'decoded_url'
07/19/2025 03:11:05 AM - Error converting news item: 'decoded_url'
07/19/2025 03:11:06 AM - Error converting news item: 'decoded_

### Get wiki and profile infos via Wikipedia

In [3]:
from tqdm import tqdm

wikiCollector = WikipediaCollector()
profiles = []

for name in tqdm(trending_celebs, desc="Collecting celeb profiles", unit="celeb"):
    saved = wikiCollector.save_wiki(name)
    if not saved:
        continue
    
    profile = wikiCollector.profile(name)
    if profile:
        profiles.append(profile)

In [4]:
from dataclasses import asdict

profile_dicts = [asdict(p) for p in profiles]
celeb_profiles = pd.DataFrame(profile_dicts)
celeb_profiles.to_csv('../documents/profiles.csv')
celeb_profiles

""


### Collect Articles for Each “Hot Star”

In [ ]:
from collectors.google_news import CelebrityGoogleNewsCollector
from newspaper import Article

news_collector = CelebrityGoogleNewsCollector()
news_collected = []

base_path = 'articles'


def serialize_relation(rel):
    data = asdict(rel)
    data['relationship'] = data['relationship'].value
    data['start_yr'] = rel.start_yr.isoformat() if rel.start_yr else None
    data['end_yr'] = rel.end_yr.isoformat() if rel.end_yr else None
    return data


for p in profiles:
    news_list = await news_collector.get_news(p.name, max_results=10)
    is_first_married = bool(
        p.spouse) and p.spouse[0].relationship == RelationshipStatus.MARRIED
    relationships = [serialize_relation(rel)
                     for rel in p.spouse] if p.spouse else []

    base_md = {
        'celeb': p.name,
        'occupatin': p.occupation,
        'spouse': relationships[0] if is_first_married else None,
        'ex_relations': relationships[1:] if is_first_married else relationships
    }

    for news in news_list:
        await news_collector.save_article(
            url=news.url, 
            metadata={'source': news.publisher, **base_md})